In [1]:
##IMPORT SECTION
import pandas
import cx_Oracle as orcl
import getpass
import os

In [2]:
DF_DIR = '/Users/cartik/Avillach-Lab/PTEN_Hamartoma_Syndrome_IDDRC/df/'
PHTS_FILE = 'bch-phts.csv'
ASD_FILE = 'bch-asd.csv'

In [3]:
##CONNECT TO DATABASE USING USER INPUT
def ConnectToDB():
    uid = input("Enter username (eCommons ID):")
    pwd = getpass.getpass("Enter password:")
    portnum = '51531'
    conn_str = uid + '/' + pwd + '@localhost:' + portnum + "/ORCL"
    conn = orcl.connect(conn_str)
    return conn

In [4]:
##THE QUERY -- "NOTE:564492681" and "NOTE:465755133" are CONCEPT_CD for 'PTEN Del/Dup' and 'SMC-PTEN'
##WE FIND ALL THE ICD-9 CODES ASSOCIATED WITH EACH PATIENT WHO HAS REPORTED 'PTEN Del/Dup' OR 'SMC-PTEN'
def QueryDbForData(Conn, AsdOrPhts='phts'):
    if AsdOrPhts == 'phts':
        query = """WITH c1 AS ( 
            SELECT DISTINCT patient_num FROM asd_gi.observation_fact WHERE concept_cd IN 
                ('NOTE:564492681', 'NOTE:465755133')
            )
            SELECT DISTINCT 
            obf.patient_num AS patient, 
            obf.concept_cd AS code, 
            cd.name_char AS condition
            FROM
            asd_gi.observation_fact obf,
            asd_gi.concept_dimension cd
            WHERE
            obf.concept_cd = cd.concept_cd AND 
            regexp_like(cd.concept_cd, '^ICD9') AND 
            obf.patient_num IN (
            SELECT patient_num FROM c1
            )"""
    else:
        query = """SELECT DISTINCT 
            patient_num, concept_cd 
            FROM 
            asd_gi.observation_fact 
            WHERE 
            regexp_like(concept_cd, '^ICD9')"""
    res = pandas.read_sql(query, conn)
    return res

In [5]:
#GET DATAFRAME WITH AUTISM PATIENTS
if os.path.isfile(DF_DIR + ASD_FILE):
    asd_df = pandas.read_csv(DF_DIR + ASD_FILE)
else:
    print("Connecting to DB for Autism Data")
    conn = ConnectToDB()
    asd_df = QueryDbForData(conn, 'asd')
    conn.close()

In [6]:
#GET DATAFRAME WITH PHTS PATIENTS
if os.path.isfile(DF_DIR + PHTS_FILE):
    phts_df = pandas.read_csv(DF_DIR + PHTS_FILE)
else:
    print("Connecting to DB for PHTS Data")
    conn = ConnectToDB()
    phts_df = QueryDbForData(conn, 'phts')
    conn.close()

In [7]:
##PERSISTENCE!! THESE QUERIES TAKE OVER 20 MINUTES TO COMPLETE
if os.path.isfile(DF_DIR + PHTS_FILE) == 0:
    phts_df.to_csv(DF_DIR + PHTS_FILE)
if os.path.isfile(DF_DIR + ASD_FILE) == 0:
    asd_df.to_csv(DF_DIR + ASD_FILE)

In [8]:
phts_stats_df = phts_df.groupby(['CONDITION','CODE']).size().reset_index().rename(columns={0:'PATIENT_COUNT'}).sort_values(by='PATIENT_COUNT', ascending = False)
#df.groupby([‘CONDITION’,‘CODE’]).size().reset_index().rename(columns={0:‘count’}).sort_values(by=‘count’, ascending = False)

In [9]:
phts_stats_df.head(10)

,CONDITION,CODE,PATIENT_COUNT
396,"Infantile autism, current or active state",ICD9:299.00,29
96,"Autistic disorder, current or active state",ICD9:299.00,29
1005,Unspecified delay in development,ICD9:315.9,25
66,Anomalies of skull and face bones,ICD9:756.0,25
175,Congenital anomalies of skull and face bones,ICD9:756.0,25
1029,Unspecified hearing loss,ICD9:389.9,17
439,"Laboratory examination, unspecified",ICD9:V72.60,14
747,Other specified congenital anomalies of brain,ICD9:742.4,14
296,Expressive language disorder,ICD9:315.31,13
232,Developmental language disorder,ICD9:315.31,13


In [10]:
##FIND THE TOP 40 CONDITIONS
Top50Conditions = pandas.DataFrame(phts_stats_df.nlargest(50, 'PATIENT_COUNT'))

In [11]:
uniq_codes = Top50Conditions['CODE'].unique()

In [12]:
def GenerateCodePairs(codes):
    import itertools as itools
    combs = list(itools.combinations(codes, 2))
    return combs

In [13]:
def ComputePatientCountsWithCombinationOfConditions(df, cond1, cond2):
    p1 = pandas.DataFrame(df[df.CODE == cond1].PATIENT.unique())
    p2 = pandas.DataFrame(df[df.CODE == cond2].PATIENT.unique())
    frames = [p1, p2]
    union = pandas.concat(frames).drop_duplicates()
    intersect = pandas.merge(p1, p2, how='inner')
    return len(union), len(intersect)

In [14]:
combinations = GenerateCodePairs(uniq_codes)

In [15]:
list = []
cols = ['CODE_1', 'CODE_2', 'PATIENTS_WITH_EITHER', 'PATIENTS_WITH_BOTH']

In [16]:
for index in range(len(combinations)):
    c1, c2 = combinations[index][0], combinations[index][1]
    Either, Both = ComputePatientCountsWithCombinationOfConditions(phts_df, c1, c2)
    list.append([c1, c2, Either, Both])

comb_df = pandas.DataFrame(list, columns = cols)

In [17]:
result = comb_df.sort_values(by='PATIENTS_WITH_BOTH', ascending=False)

In [18]:
result = result[result.CODE_1 != 'ICD9:299.00']

In [19]:
result.head(30)

,CODE_1,CODE_2,PATIENTS_WITH_EITHER,PATIENTS_WITH_BOTH
39,ICD9:315.9,ICD9:756.0,35,15
40,ICD9:315.9,ICD9:389.9,31,11
44,ICD9:315.9,ICD9:V82.79,28,10
42,ICD9:315.9,ICD9:742.4,29,10
77,ICD9:756.0,ICD9:389.9,32,10
78,ICD9:756.0,ICD9:V72.60,30,9
48,ICD9:315.9,ICD9:315.39,27,9
47,ICD9:315.9,ICD9:315.8,27,9
45,ICD9:315.9,ICD9:780.39,28,9
41,ICD9:315.9,ICD9:V72.60,30,9
